In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
# cols = ['col_' + str(i) for i in range(40)]

In [67]:
train_X = pd.read_csv('Day_048_data-science-london-scikit-learn/train.csv',header = None)

In [68]:
train_Y = pd.read_csv('Day_048_data-science-london-scikit-learn/trainLabels.csv',header=None)
# train_Y.columns = ['target']
# train_X['is_train'] = 1

In [69]:
test_X = pd.read_csv('Day_048_data-science-london-scikit-learn/test.csv',header = None)

In [57]:
# test_X['is_train'] = 0
# test_X['target'] = None

In [58]:
# master = train_X
# master['target'] = train_Y.target.values
# # pd.merge(train_X
# # ,train_Y, left_index=True, right_index=True)

# master = pd.concat([master,test_X])

# print(train_X.shape, test_X.shape,master.shape)

# master['target'] = pd.to_numeric(master['target'])

# # master.info()

# master.describe()

In [70]:
x_train, x_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.25, random_state=42)

# 建立模型
clf = GradientBoostingClassifier(random_state=7)

In [71]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

/Users/ttyoung/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  20 out of  27 | elapsed:    2.8s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    4.2s finished
/Users/ttyoung/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [72]:
grid_result.best_params_

{'max_depth': 5, 'n_estimators': 100}

In [73]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

y_pred_proba = clf_bestparam.predict_proba(x_test)

/Users/ttyoung/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [74]:
print(metrics.accuracy_score(y_test, y_pred))

0.892


In [75]:
metrics.log_loss(y_test, y_pred_proba)

0.32935795038860577

In [76]:
predictions = clf_bestparam.predict(test_X)

In [84]:
predictions_pd = pd.DataFrame(list(zip(list(range(1,9001)),predictions)),columns=['Id','Solution'])
# len()

In [85]:
predictions_pd.head()

,Id,Solution
0,1,1
1,2,0
2,3,0
3,4,0
4,5,0


In [86]:
predictions_pd.to_csv('predictions.csv',index=False)

## xgboost

In [59]:
from xgboost import XGBClassifier